In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from logger import Logger


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# MNIST dataset 
dataset = torchvision.datasets.MNIST(root='../../data', 
                                     train=True, 
                                     transform=transforms.ToTensor(),  
                                     download=True)

# Data loader
data_loader = torch.utils.data.DataLoader(dataset=dataset, 
                                          batch_size=100, 
                                          shuffle=True)


# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size=784, hidden_size=500, num_classes=10):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

model = NeuralNet().to(device)

logger = Logger('./logs') #Logger.__init__(self, log_dir): Create a summary writer logging to log_dir.

# Loss and optimizer
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)  

In [2]:
#from torch.utils.data.dataloader.DataLoader to torch.utils.data.dataloader._DataLoaderIter
data_iter = iter(data_loader) 
iter_per_epoch = len(data_loader)
total_step = 50000

In [3]:
# Start training
for step in range(total_step):
    
    # Reset the data_iter
    if (step+1) % iter_per_epoch == 0:
        data_iter = iter(data_loader)

    # Fetch images and labels
    images, labels = next(data_iter) #only when data_loader is converted to data_iter can "next" be used
    images, labels = images.view(images.size(0), -1).to(device), labels.to(device)
    
    # Forward pass
    outputs = model(images)
    loss = criterion(outputs, labels)
    
    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Compute accuracy
    _, argmax = torch.max(outputs, 1)
    #torch.squeeze: Returns a tensor with all the dimensions of :attr:`input` of size `1` removed.
    #.float() : equals to self.to(torch.float32), if without it:
    # get RuntimeError: Can only calculate the mean of floating types. Got Byte instead.
    accuracy = (labels == argmax.squeeze()).float().mean()

    if (step+1) % 100 == 0:
        print ('Step [{}/{}], Loss: {:.4f}, Acc: {:.2f}' 
               .format(step+1, total_step, loss.item(), accuracy.item()))

        # ================================================================== #
        #                        Tensorboard Logging                         #
        # ================================================================== #

        # 1. Log scalar values (scalar summary)
        info = { 'loss': loss.item(), 'accuracy': accuracy.item() }

        for tag, value in info.items():
            # scalar_summary(self, tag, value, step): Log a scalar variable.
            logger.scalar_summary(tag, value, step+1)

        # 2. Log values and gradients of the parameters (histogram summary)
        # torch.nn.modules.module.named_parameters(prefix='', recurse=True) method of __main__.NeuralNet instance
        #  Returns an iterator over module parameters, yielding both the
        #  name of the parameter as well as the parameter itself.
        for tag, value in model.named_parameters():
            # tag contains: fc1.weight, fc1.bias, fc2.weight, fc2.bias
            tag = tag.replace('.', '/')
            # histo_summary(tag, values, step, bins=1000) method of logger.Logger instance
            #  Log a histogram of the tensor of values.
            logger.histo_summary(tag, value.data.cpu().numpy(), step+1)
            logger.histo_summary(tag+'/grad', value.grad.data.cpu().numpy(), step+1)

        # 3. Log training images (image summary)
        info = { 'images': images.view(-1, 28, 28)[:10].cpu().numpy() }

        for tag, images in info.items():
            logger.image_summary(tag, images, step+1)

Step [100/50000], Loss: 2.2154, Acc: 0.41
Step [200/50000], Loss: 2.1110, Acc: 0.65
Step [300/50000], Loss: 2.0177, Acc: 0.76
Step [400/50000], Loss: 1.8438, Acc: 0.79
Step [500/50000], Loss: 1.6922, Acc: 0.82
Step [600/50000], Loss: 1.5949, Acc: 0.77
Step [700/50000], Loss: 1.5189, Acc: 0.75
Step [800/50000], Loss: 1.4046, Acc: 0.80
Step [900/50000], Loss: 1.2296, Acc: 0.87
Step [1000/50000], Loss: 1.0934, Acc: 0.86
Step [1100/50000], Loss: 1.0664, Acc: 0.88
Step [1200/50000], Loss: 1.0420, Acc: 0.84
Step [1300/50000], Loss: 0.9207, Acc: 0.86
Step [1400/50000], Loss: 0.8425, Acc: 0.85
Step [1500/50000], Loss: 0.8956, Acc: 0.78
Step [1600/50000], Loss: 0.8901, Acc: 0.76
Step [1700/50000], Loss: 0.7378, Acc: 0.87
Step [1800/50000], Loss: 0.7390, Acc: 0.81
Step [1900/50000], Loss: 0.6066, Acc: 0.88
Step [2000/50000], Loss: 0.7477, Acc: 0.86
Step [2100/50000], Loss: 0.5677, Acc: 0.90
Step [2200/50000], Loss: 0.5867, Acc: 0.87
Step [2300/50000], Loss: 0.5638, Acc: 0.90
Step [2400/50000], L

Step [19000/50000], Loss: 0.2430, Acc: 0.96
Step [19100/50000], Loss: 0.3173, Acc: 0.90
Step [19200/50000], Loss: 0.2299, Acc: 0.91
Step [19300/50000], Loss: 0.2688, Acc: 0.94
Step [19400/50000], Loss: 0.2585, Acc: 0.92
Step [19500/50000], Loss: 0.1499, Acc: 0.98
Step [19600/50000], Loss: 0.1107, Acc: 0.97
Step [19700/50000], Loss: 0.1931, Acc: 0.94
Step [19800/50000], Loss: 0.1426, Acc: 0.95
Step [19900/50000], Loss: 0.1790, Acc: 0.95
Step [20000/50000], Loss: 0.2355, Acc: 0.93
Step [20100/50000], Loss: 0.1467, Acc: 0.98
Step [20200/50000], Loss: 0.2687, Acc: 0.92
Step [20300/50000], Loss: 0.1625, Acc: 0.95
Step [20400/50000], Loss: 0.3327, Acc: 0.90
Step [20500/50000], Loss: 0.1575, Acc: 0.95
Step [20600/50000], Loss: 0.1717, Acc: 0.95
Step [20700/50000], Loss: 0.2281, Acc: 0.92
Step [20800/50000], Loss: 0.2486, Acc: 0.94
Step [20900/50000], Loss: 0.1954, Acc: 0.94
Step [21000/50000], Loss: 0.1758, Acc: 0.96
Step [21100/50000], Loss: 0.1686, Acc: 0.97
Step [21200/50000], Loss: 0.1429

Step [37700/50000], Loss: 0.2042, Acc: 0.95
Step [37800/50000], Loss: 0.2141, Acc: 0.94
Step [37900/50000], Loss: 0.1211, Acc: 0.98
Step [38000/50000], Loss: 0.1230, Acc: 0.97
Step [38100/50000], Loss: 0.1292, Acc: 0.96
Step [38200/50000], Loss: 0.2580, Acc: 0.96
Step [38300/50000], Loss: 0.1367, Acc: 0.95
Step [38400/50000], Loss: 0.1486, Acc: 0.94
Step [38500/50000], Loss: 0.0844, Acc: 0.98
Step [38600/50000], Loss: 0.1249, Acc: 0.98
Step [38700/50000], Loss: 0.1288, Acc: 0.97
Step [38800/50000], Loss: 0.0572, Acc: 0.99
Step [38900/50000], Loss: 0.1843, Acc: 0.95
Step [39000/50000], Loss: 0.2046, Acc: 0.95
Step [39100/50000], Loss: 0.1196, Acc: 0.97
Step [39200/50000], Loss: 0.1660, Acc: 0.97
Step [39300/50000], Loss: 0.1210, Acc: 0.98
Step [39400/50000], Loss: 0.1802, Acc: 0.95
Step [39500/50000], Loss: 0.1546, Acc: 0.96
Step [39600/50000], Loss: 0.0933, Acc: 0.97
Step [39700/50000], Loss: 0.1251, Acc: 0.97
Step [39800/50000], Loss: 0.2713, Acc: 0.91
Step [39900/50000], Loss: 0.0927

In [4]:
# !pip install -r requirements.txt

In [5]:
# !python main.py

In [6]:
# !tensorboard --logdir='./logs' --port=6006